**제출 시 파일 제목에 `이름`을 수정하여 제출할 것!**

In [25]:
!pip install selenium
!pip install webdriver-manager

import re
import requests

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

options = Options()

options.add_argument("--headless")
options.add_argument('--no-sandbox')

# Q.1


### 1. 문자열 "He has 12 apples and 5 oranges." 에서 숫자만 추출하는 정규 표현식을 작성하고, `search`, `findall`, `finditer` 세가지를 모두 사용하여 추출해보고 차이점을 설명하시오.

In [26]:
text = "He has 12 apples and 5 oranges."
pattern = r'\d+'

m_search = re.search(pattern, text)
if m_search:
  print(m_search.group())
  print(m_search.span())
print()

all_nums = re.findall(pattern, text)
print(all_nums)
print()

for m in re.finditer(pattern, text):
  print(m.group(), m.span())

12
(7, 9)

['12', '5']

12 (7, 9)
5 (21, 22)


search는 문자열에 첫 번째 매치 하나만 찾는다. findall은 모든 매치의 값을 리스트로 만들고 finditer는 모든 매치를 객체로 반환한다.

### 2. 가장 많은 수를 가지고 있는 과일이 무엇인지 출력하시오. (힌트: grouping을 사용할 것)

In [27]:
pattern = r'(\d+)\s+([a-zA-Z]+)'
maxcnt = -1
maxf = ''

for m in re.finditer(pattern, text):
  cnt = int(m.group(1))
  f = m.group(2)

  if cnt > maxcnt:
    maxcnt = cnt
    maxf = f

print(maxf, maxcnt)

apples 12


# Q.2

### 1. 이메일 주소가 유효한 이메일 형태로 작성되어 있는지 확인하는 함수 `is_valid_email`를 생성하고 유효한 이메일일 경우 이메일 주소를, 아닐 경우 `유효한 이메일 주소가 아닙니다.`를 출력하시오.

**Example**
```python
def is_valid_email(email):
  ...

address1 = 'test@gmail.com'
address2 = 'test.gmail.com'

print(is_valid_email(address1))
>> test@gmail.com
print(is_valid_email(address2))
>> 유효한 이메일 주소가 아닙니다.
```

In [28]:
def is_valid_email(email):
  pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'

  if re.match(pattern, email):
    return email
  else:
    return "유효한 이메일 주소가 아닙니다."

address1 = 'test@gmail.com'
address2 = 'test.gmail.com'

print(is_valid_email(address1))
print(is_valid_email(address2))

test@gmail.com
유효한 이메일 주소가 아닙니다.


### 2. Q2.1번 문제에 이어서 유효한 이메일일 경우, 해당 이메일의 아이디와 도메인을 출력하시오.:

**Example**

```python
def is_valid_email2(email):
  ...

address1 = 'test@gmail.com'
address2 = 'test.gmail.com'

print(is_valid_email2(address1))
>> ID: test, Domain: gmail.com
print(is_valid_email2(address2))
>> 유효한 이메일 주소가 아닙니다.

```

In [29]:
for addr in [address1, address2]:
    result = is_valid_email(addr)

    if result == "유효한 이메일 주소가 아닙니다.":
        print(result)
    else:
        user_id, domain = result.split('@', 1)
        print(f"ID: {user_id}, Domain: {domain}")

ID: test, Domain: gmail.com
유효한 이메일 주소가 아닙니다.


# Q.3

### 정규표현식을 사용해서 html의 문자열을 파싱하는 beautifulsoup의 find()와 get_text() function을 갖는 ParserWithRegex라는 클래스를 구현해보시오.

**Example**
```python
class ParserWithRegex:
    def __init__(self, html):
        self.source = html

    def find(self, tag):
      ...

    def get_text(self, html_fragment):
      ...


html = """
<html>
<body>
<div>Hello <b>World</b>!</div>
<p>This is a paragraph.</p>
<div class=test>Another <i>div</i>.</div>
</body>
</html>
"""

parser = ParserWithRegex(html)

div_content = parser.find('div')
print(div_content)
>> Hello <b>World</b>!

text = parser.get_text(div_content)
print(text)
>> Hello World!
```

In [30]:
class ParserWithRegex:
    def __init__(self, html):
        self.source = html

    def find(self, tag):
        pattern = rf'<{tag}\b[^>]*>(.*?)</{tag}>'
        m = re.search(pattern, self.source, re.DOTALL)

        if m:
            return m.group(1).strip()
        return None

    def get_text(self, html_fragment):
        if html_fragment is None:
            return None

        text = re.sub(r'<[^>]+>', '', html_fragment)
        return text.strip()

html = """
<html>
<body>
<div>Hello <b>World</b>!</div>
<p>This is a paragraph.</p>
<div class=test>Another <i>div</i>.</div>
</body>
</html>
"""
parser = ParserWithRegex(html)

div_content = parser.find('div')
print(div_content)

text = parser.get_text(div_content)
print(text)


Hello <b>World</b>!
Hello World!


# Q.4

### 공공데이터 포털에서 [소상공인시장진흥공단_상가(상권)정보_API](https://www.data.go.kr/data/15012005/openapi.do)를 활용하여 다음 질문들에 답하시오.

> 서비스키는 각자의 서비스키를 입력하셔야 합니다. 다른 학생과 중복시 카피로 간주합니다.
>
> **[힌트]**
>
> 마이페이지>데이터 활용>Open API>활용신청 현황>사용하고자 하는 API>상세설명에 들어가시면 참고문서로 활용가이드를 확인할 수 있습니다. 활용가이드와 업종분류표를 잘 확인하시고 진행하시길 바랍니다.



#### 1. 행정구역 조회 오퍼레이션(상세 기능)을 통해 서울특별시 성북구 안암동의 시도, 시군구, 행정동 코드를 구하시오.

In [31]:
BASE = "https://apis.data.go.kr/B553077/api/open/sdsc2"
KEY = "f95e1e5448f207d52930fe8b951ed6ee3639cea92ab99c38ec71cbc6cfa25ac4"

def fetch_items(catId):
    url = f"{BASE}/baroApi?serviceKey={KEY}&resId=dong&catId={catId}&type=json"
    return requests.get(url).json()["body"]["items"]

def get_area_code():
    sido_items = fetch_items("mega")
    sido = next(item for item in sido_items if item["ctprvnNm"] == "서울특별시")
    sido_cd = sido["ctprvnCd"]

    sigungu_items = fetch_items("cty")
    sigungu = next(item for item in sigungu_items
                   if item["ctprvnCd"] == sido_cd and item["signguNm"] == "성북구")
    sigungu_cd = sigungu["signguCd"]

    dong_items = fetch_items("admi")
    dong = next(item for item in dong_items
                if item["signguCd"] == sigungu_cd and item["adongNm"] == "안암동")
    dong_cd = dong["adongCd"]

    print("시도코드:", sido_cd)
    print("시군구코드:", sigungu_cd)
    print("행정동코드:", dong_cd)

    return sido_cd, sigungu_cd, dong_cd

get_area_code()


시도코드: 11
시군구코드: 11290
행정동코드: 11290600


('11', '11290', '11290600')

#### 2. 서울특별시 성북구 안암동의 음식점업 총 수를 구하시오.

In [32]:
def count_food_in_anam():
    url = (
        f"{BASE}/storeListInDong"
        f"?serviceKey={KEY}"
        f"&pageNo=1"
        f"&numOfRows=1000"
        f"&divId=adongCd"
        f"&key=11290600"
        f"&indsLclsCd=I2"
        f"&type=json"
    )
    res = requests.get(url).json()
    items = res["body"]["items"]
    print("안암동 음식점 총 수:", len(items))
    return len(items)

count_food_in_anam()


안암동 음식점 총 수: 468


468

#### 3. 자신이 안암에서 가장 좋아하는 맛집을 찾아 상호명, 상권업종중분류명, 상권업종소분류명, 도로명주소를 출력하시오.

In [33]:
def food_in_anam():
    url = (
        f"{BASE}/storeListInDong"
        f"?serviceKey={KEY}"
        f"&pageNo=1"
        f"&numOfRows=1000"
        f"&divId=adongCd"
        f"&key=11290600"
        f"&indsLclsCd=I2"
        f"&type=json"
    )
    res = requests.get(url).json()
    items = res["body"]["items"]

    return len(items), res

cnt, index = food_in_anam()

def find_gopumkong(index):
    items = index["body"]["items"]
    found = False

    for item in items:
        name = item.get("bizesNm", "")
        if "고품콩" in name:
            found = True
            print("상호명:", name)
            print("상권업종중분류명:", item.get("indsMclsNm", "중분류 없음"))
            print("상권업종소분류명:", item.get("indsSclsNm", "소분류 없음"))
            print("도로명주소:", item.get("rdnmAdr", "도로명주소 없음"))


find_gopumkong(index)

상호명: 고품콩
상권업종중분류명: 한식
상권업종소분류명: 돼지고기 구이/찜
도로명주소: 서울특별시 성북구 안암로 87-5


# Q.5

### 다음 HTML 구조를 가진 웹 페이지가 있다고 가정하자.
```
<!DOCTYPE html>
<html>
<head>
    <title>Sample Web Page</title>
</head>
<body>
    <h1>Welcome to the Sample Web Page</h1>
    <div id="author">
        Author: Hyeongwon Kang
    </div>
    <div id="content">
        <p>This is a paragraph in the web page.</p>
        <p>This is another paragraph, with a <a href="https://www.example.com">link</a>.</p>
    </div>
    <ul id="items">
        <li class="item">Item 1</li>
        <li class="item">Item 2</li>
        <li class="item">Item 3</li>
    </ul>
</body>
</html>
```


#### 1. 페이지의 제목(`<title>`)을 추출하여라.

In [34]:
html = """
<!DOCTYPE html>
<html>
<head>
    <title>Sample Web Page</title>
</head>
<body>
    <h1>Welcome to the Sample Web Page</h1>
    <div id="author">
        Author: Hyeongwon Kang
    </div>
    <div id="content">
        <p>This is a paragraph in the web page.</p>
        <p>This is another paragraph, with a <a href="https://www.example.com">link</a>.</p>
    </div>
    <ul id="items">
        <li class="item">Item 1</li>
        <li class="item">Item 2</li>
        <li class="item">Item 3</li>
    </ul>
</body>
</html>
"""

from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")
title = soup.title.string

print(title)


Sample Web Page


#### 2. "content"라는 ID를 가진 `<div>` 태그의 모든 텍스트를 추출하여라.

In [35]:
content_div = soup.find("div", id="content")
content_text = content_div.get_text(" ", strip=True)
print(content_text)

This is a paragraph in the web page. This is another paragraph, with a link .


#### 3. "items"라는 ID를 가진 `<ul>` 태그 내에 있는 모든 항목(`<li>`)의 텍스트를 리스트로 만들어 반환하여라.

In [36]:
items_ul = soup.find("ul", id="items")
li_tags = items_ul.find_all("li")
li_texts = [li.get_text(strip=True) for li in li_tags]
print(li_texts)

['Item 1', 'Item 2', 'Item 3']


#### 4. CSS를 사용해서 link를 추출하여라.

뷁

# Q.6

### 다음 기사의 본문에서 강필성 교수님이 몇번 언급되는지 구하시오.

https://www.seoul.co.kr/news/newsView.php?id=20200108500123&wlog_tag3=naver

> 기사 본문을 크롤링하고 정규표현식을 사용합니다.
>
> '강 교수'도 강필성 교수님 언급으로 카운트 되어야 합니다.
>
> 패턴은 단 한번만 사용 가능합니다.

In [42]:
import requests, re
from bs4 import BeautifulSoup

URL = "https://www.seoul.co.kr/news/newsView.php?id=20200108500123"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "ko-KR,ko;q=0.9",
    "Referer": "https://www.google.com"
}

res = requests.get(URL, headers=headers)
res.raise_for_status()

res.encoding = "utf-8"

soup = BeautifulSoup(res.text, "html.parser")

article_div = soup.select_one("div.viewContent.body18.color700")

text = article_div.get_text(" ", strip=True)

pattern = re.compile(r"강(필성)?\s*교수")
count = len(pattern.findall(text))
print("언급 횟수:", count)


언급 횟수: 6


# Q.7

### 다음 기사에서 모든 댓글들을 크롤링하여 리스트 형태로 만들고, 댓글 리스트와 댓글 갯수를 출력하시오.

https://n.news.naver.com/article/138/0002161591?ntype=RANKING

> 본 기사 페이지에서 시작해야합니다.
>
> selenium을 활용해서 댓글 더보기 버튼을 눌러 댓글들을 load 합니다.
>
> **힌트**
>
> 댓글 더보기 버튼을 클릭하여 모든 댓글들을 불러옵니다.
>
> 댓글 더보기 버튼이 클릭되지 않으면 모든 댓글들이 load 된 상태입니다.
>
> **추천 방법 (꼭 이렇게 하지 않으셔도 됩니다.)**
>
> 1) 전체 댓글 수를 크롤링하고 댓글이 전체 댓글 수 만큼 출력될 때까지만 더보기 클릭하기
>
> 2) try, except 코드를 사용해서 더보기 버튼이 클릭되지 않는 상황에서 종료하기

**try, except 설명**
- `try`와 `except`는 파이썬에서 예외 처리를 위해 사용되는 구문이다. 이 구문을 사용하면 코드 실행 중에 발생할 수 있는 오류(예외)를 처리하고, 프로그램이 예외로 인해 갑자기 중단되는 것을 방지할 수 있다. `try` 블록 안의 코드를 실행하다가 오류가 발생하면 즉시 그 블록을 중단하고 `except` 블록으로 이동한다.
```python
try:
    예외가 발생할 수 있는 코드
except:
    Error가 발생했을 때 실행될 코드
```